# Delta calculator

A simple delta calculator

In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes.greeks.numerical as greeks
import py_vollib.black_scholes.implied_volatility as iv
import py_vollib.black_scholes as bs
from typing import Tuple

In [2]:
# parameters
spot:float = 17000
strike:float = 17000
risk_free_rate:float = 0.1
iv_range: Tuple[float, float] = (0.15,0.25)
step:float = 50 
de:int = 8 # days to expiry



In [7]:
greeks.delta?

Signature: greeks.delta(flag, S, K, t, r, sigma)
Docstring:
Return Black-Scholes delta of an option.

:param S: underlying asset price
:type S: float
:param K: strike price
:type K: float
:param sigma: annualized standard deviation, or volatility
:type sigma: float
:param t: time to expiration in years
:type t: float
:param r: risk-free interest rate
:type r: float
:param flag: 'c' or 'p' for call or put.
:type flag: str
File:      ~/.local/lib/python3.7/site-packages/py_vollib/black_scholes/greeks/numerical.py
Type:      function


In [8]:
kwargs = {
    'flag': 'c',
    'S': spot,
    'K': strike,
    't': de/360,
    'r': risk_free_rate
    
}

In [9]:
ivs = np.linspace(iv_range[0], iv_range[-1], 10)
strikes = set() 
for i in range(5):
    strikes.add((int(spot/step)*step) + i*step)
    strikes.add((int(spot/step)*step) - i*step)
spot_vals = np.linspace(spot*0.98,spot*1.02,10)
    


In [10]:
calls = []
puts = []
for sigma in ivs:
    for strk in strikes:
        kwargs.update({'sigma': sigma, 'K': strk, 'S': spot, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, strk, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, strk, price, grk))
        

## Call option Delta (by strike)

**Call option delta for a fixed spot price with various strikes and implied volatility**

In [11]:
df1 = pd.DataFrame(calls, columns=['iv', 'strike', 'price', 'greek']).round(4)
df1_pivot = df1.pivot(index='iv', columns='strike', values='greek').round(4)
df1_pivot

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.1500,0.7389,0.6939,0.6459,0.5957,0.5440,0.4917,0.4397,0.3889,0.3400
0.1611,0.7248,0.6821,0.6369,0.5899,0.5416,0.4929,0.4445,0.3970,0.3511
0.1722,0.7124,0.6717,0.6290,0.5848,0.5396,0.4940,0.4487,0.4041,0.3609
0.1833,0.7013,0.6625,0.6220,0.5803,0.5378,0.4950,0.4524,0.4104,0.3696
0.1944,0.6913,0.6543,0.6159,0.5764,0.5363,0.4959,0.4557,0.4161,0.3774
0.2056,0.6823,0.6469,0.6104,0.5730,0.5350,0.4968,0.4588,0.4212,0.3844
0.2167,0.6741,0.6403,0.6055,0.5699,0.5339,0.4976,0.4615,0.4258,0.3908
0.2278,0.6667,0.6344,0.6011,0.5672,0.5328,0.4984,0.4640,0.4300,0.3966
0.2389,0.6600,0.6290,0.5971,0.5647,0.5320,0.4991,0.4663,0.4339,0.4020


## Put option Delta (by strike)
**Put option delta for a fixed spot price with various strikes and implied volatility**

In [12]:
df2 = pd.DataFrame(puts, columns=['iv', 'strike', 'price', 'greek']).round(4)
df2_pivot = df2.pivot(index='iv', columns='strike', values='greek').round(4)
df2_pivot

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.1500,-0.2611,-0.3061,-0.3541,-0.4043,-0.4560,-0.5083,-0.5603,-0.6111,-0.6600
0.1611,-0.2752,-0.3179,-0.3631,-0.4101,-0.4584,-0.5071,-0.5555,-0.6030,-0.6489
0.1722,-0.2876,-0.3283,-0.3710,-0.4152,-0.4604,-0.5060,-0.5513,-0.5959,-0.6391
0.1833,-0.2987,-0.3375,-0.3780,-0.4197,-0.4622,-0.5050,-0.5476,-0.5896,-0.6304
0.1944,-0.3087,-0.3457,-0.3841,-0.4236,-0.4637,-0.5041,-0.5443,-0.5839,-0.6226
0.2056,-0.3177,-0.3531,-0.3896,-0.4270,-0.4650,-0.5032,-0.5412,-0.5788,-0.6156
0.2167,-0.3259,-0.3597,-0.3945,-0.4301,-0.4661,-0.5024,-0.5385,-0.5742,-0.6092
0.2278,-0.3333,-0.3656,-0.3989,-0.4328,-0.4672,-0.5016,-0.5360,-0.5700,-0.6034
0.2389,-0.3400,-0.3710,-0.4029,-0.4353,-0.4680,-0.5009,-0.5337,-0.5661,-0.5980


# Combined Delta - fixed spot

In [13]:
pvt = (df1_pivot + df2_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.15,0.477800,0.387800,0.291800,0.191400,0.088000,-0.016600,-0.120600,-0.222200,-0.320000
0.1611,0.449600,0.364200,0.273800,0.179800,0.083200,-0.014200,-0.111000,-0.206000,-0.297800
0.1722,0.424800,0.343400,0.258000,0.169600,0.079200,-0.012000,-0.102600,-0.191800,-0.278200
0.1833,0.402600,0.325000,0.244000,0.160600,0.075600,-0.010000,-0.095200,-0.179200,-0.260800
0.1944,0.382600,0.308600,0.231800,0.152800,0.072600,-0.008200,-0.088600,-0.167800,-0.245200
0.2056,0.364600,0.293800,0.220800,0.146000,0.070000,-0.006400,-0.082400,-0.157600,-0.231200
0.2167,0.348200,0.280600,0.211000,0.139800,0.067800,-0.004800,-0.077000,-0.148400,-0.218400
0.2278,0.333400,0.268800,0.202200,0.134400,0.065600,-0.003200,-0.072000,-0.140000,-0.206800
0.2389,0.320000,0.258000,0.194200,0.129400,0.064000,-0.001800,-0.067400,-0.132200,-0.196000


In [14]:
calls = []
puts = []
for sigma in ivs:
    for spt in spot_vals:
        kwargs.update({'sigma': sigma, 'S': spt, 'K':strike, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, spt, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, spt, price, grk))

## Call option delta (by spot)

**Call option delta for a fixed strike price with various spot prices and implied volatility**


In [15]:
df3 = pd.DataFrame(calls, columns=['iv', 'spot', 'price', 'greek']).round(4)
df3['spot'] = df3.spot.astype(int)
df3_pivot = df3.pivot(index='iv', columns='spot', values='greek').round(4)
df3_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1500,0.2139,0.2774,0.3486,0.4252,0.5044,0.5831,0.6583,0.7273,0.7884,0.8404
0.1611,0.2307,0.2918,0.3592,0.4309,0.5047,0.5781,0.6485,0.7138,0.7725,0.8236
0.1722,0.2459,0.3046,0.3685,0.4360,0.5051,0.5737,0.6399,0.7019,0.7582,0.8080
0.1833,0.2597,0.3161,0.3768,0.4405,0.5054,0.5699,0.6324,0.6912,0.7453,0.7938
0.1944,0.2723,0.3265,0.3843,0.4445,0.5057,0.5666,0.6257,0.6817,0.7336,0.7806
0.2056,0.2838,0.3359,0.3910,0.4481,0.5061,0.5637,0.6197,0.6731,0.7229,0.7686
0.2167,0.2944,0.3444,0.3971,0.4514,0.5064,0.5611,0.6143,0.6653,0.7132,0.7574
0.2278,0.3041,0.3522,0.4026,0.4544,0.5067,0.5587,0.6095,0.6583,0.7044,0.7472
0.2389,0.3131,0.3594,0.4077,0.4571,0.5071,0.5567,0.6052,0.6519,0.6963,0.7377


## Put option delta (by spot)

**Put option delta for a fixed strike price with various spot prices and implied volatility**


In [16]:
df4 = pd.DataFrame(puts, columns=['iv', 'spot', 'price', 'greek']).round(4)
df4['spot'] = df4.spot.astype(int)
df4_pivot = df4.pivot(index='iv', columns='spot', values='greek').round(4)
df4_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1500,-0.7861,-0.7226,-0.6514,-0.5748,-0.4956,-0.4169,-0.3417,-0.2727,-0.2116,-0.1596
0.1611,-0.7693,-0.7082,-0.6408,-0.5691,-0.4953,-0.4219,-0.3515,-0.2862,-0.2275,-0.1764
0.1722,-0.7541,-0.6954,-0.6315,-0.5640,-0.4949,-0.4263,-0.3601,-0.2981,-0.2418,-0.1920
0.1833,-0.7403,-0.6839,-0.6232,-0.5595,-0.4946,-0.4301,-0.3676,-0.3088,-0.2547,-0.2062
0.1944,-0.7277,-0.6735,-0.6157,-0.5555,-0.4943,-0.4334,-0.3743,-0.3183,-0.2664,-0.2194
0.2056,-0.7162,-0.6641,-0.6090,-0.5519,-0.4939,-0.4363,-0.3803,-0.3269,-0.2771,-0.2314
0.2167,-0.7056,-0.6556,-0.6029,-0.5486,-0.4936,-0.4389,-0.3857,-0.3347,-0.2868,-0.2426
0.2278,-0.6959,-0.6478,-0.5974,-0.5456,-0.4933,-0.4413,-0.3905,-0.3417,-0.2956,-0.2528
0.2389,-0.6869,-0.6406,-0.5923,-0.5429,-0.4929,-0.4433,-0.3948,-0.3481,-0.3037,-0.2623


# Combined Delta - fixed strike

Combined delta for a specific strike but with different volatility and spot prices

In [17]:
pvt = (df3_pivot + df4_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.15,-0.572200,-0.445200,-0.302800,-0.149600,0.008800,0.166200,0.316600,0.454600,0.576800,0.680800
0.1611,-0.538600,-0.416400,-0.281600,-0.138200,0.009400,0.156200,0.297000,0.427600,0.545000,0.647200
0.1722,-0.508200,-0.390800,-0.263000,-0.128000,0.010200,0.147400,0.279800,0.403800,0.516400,0.616000
0.1833,-0.480600,-0.367800,-0.246400,-0.119000,0.010800,0.139800,0.264800,0.382400,0.490600,0.587600
0.1944,-0.455400,-0.347000,-0.231400,-0.111000,0.011400,0.133200,0.251400,0.363400,0.467200,0.561200
0.2056,-0.432400,-0.328200,-0.218000,-0.103800,0.012200,0.127400,0.239400,0.346200,0.445800,0.537200
0.2167,-0.411200,-0.311200,-0.205800,-0.097200,0.012800,0.122200,0.228600,0.330600,0.426400,0.514800
0.2278,-0.391800,-0.295600,-0.194800,-0.091200,0.013400,0.117400,0.219000,0.316600,0.408800,0.494400
0.2389,-0.373800,-0.281200,-0.184600,-0.085800,0.014200,0.113400,0.210400,0.303800,0.392600,0.475400
